In [50]:
#Importamos la librería pandas que necesitamos para la lectura, conversión y limpieza de los datos.
import pandas as pd
import numpy as np
import re 

pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames
# pd.set_option('display.max_rows', None) # para poder visualizar todas las filas de los DataFrames

In [51]:
#Abrimos el csv y lo convertimos a un DataFrame.
df_HR = pd.read_csv('HR_RAW_DATA.csv', index_col= 0)
df_HR.head(10)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,employeecount,employeenumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NUMCOMPANIESWORKED,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TOTALWORKINGYEARS,TrainingTimesLastYear,WORKLIFEBALANCE,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YEARSWITHCURRMANAGER,SameAsMonthlyIncome,DateBirth,Salary,RoleDepartament,NUMBERCHILDREN,RemoteWork
0,51,No,NaN,"684,0$",NaN,6,3,NaN,1,"162,0",1,0,51,3,5,resEArch DIREcToR,3,NaN,"19537,0",6462,7,Y,No,13,"3,0",3,NaN,0,NaN,5,"3,0",20,NaN,15,15,"19537,0",1972,1000000000$,NaN,NaN,Yes
1,52,No,NaN,"699,0$",NaN,1,4,Life Sciences,1,"259,0",3,0,65,2,5,ManAGeR,3,NaN,"19999,0",5678,0,NaN,NaN,14,"3,0",1,NaN,1,"34,0",5,"3,0",33,NaN,11,9,"19999,0",1971,1000000000$,NaN,NaN,1
2,42,No,travel_rarely,"532,0$",Research & Development,4,2,Technical Degree,1,"319,0",3,0,58,3,5,ManaGER,4,Married,"19232,0",4933,1,NaN,No,11,"3,0",4,NaN,0,"22,0",3,NaN,22,NaN,11,15,"19232,0",1981,1000000000$,ManaGER - Research & Development,NaN,1
3,47,No,travel_rarely,"359,0$",NaN,2,4,Medical,1,NaN,1,1,82,3,4,ReseArCH DIrECtOr,3,Married,"17169,0",26703,3,Y,NaN,19,"3,0",2,NaN,2,NaN,2,NaN,20,NaN,5,6,"17169,0",1976,1000000000$,NaN,NaN,False
4,46,No,NaN,"1319,0$",NaN,3,3,Technical Degree,1,NaN,1,1,45,4,4,sAleS EXECUtIve,1,Divorced,NaN,7739,2,Y,No,12,"3,0",4,NaN,1,NaN,5,"3,0",19,NaN,2,8,NaN,1977,1000000000$,NaN,NaN,0
5,48,No,NaN,"117,0$",Research & Development,22,3,Medical,1,"1900,0",4,1,58,3,4,MANAger,4,NaN,"17174,0",2437,3,NaN,No,11,"3,0",2,NaN,1,NaN,3,"3,0",22,NaN,4,7,"17174,0",1975,1000000000$,MANAger - Research & Development,NaN,Yes
6,59,No,NaN,"1435,0$",NaN,25,3,Life Sciences,1,"81,0",1,1,99,3,3,Sales ExeCutIVe,1,NaN,NaN,2354,7,Y,NaN,11,"3,0",4,NaN,0,"28,0",3,"2,0",21,NaN,7,9,NaN,1964,1000000000$,NaN,NaN,True
7,42,No,travel_rarely,"635,0$",NaN,1,1,NaN,1,"387,0",2,0,99,3,2,Sales eXEcUTiVe,3,Married,NaN,24532,1,NaN,No,25,"4,0",3,NaN,0,"20,0",3,"3,0",20,NaN,11,6,NaN,1981,1000000000$,NaN,NaN,0
8,41,No,NaN,"1276,0$",NaN,2,5,NaN,1,NaN,2,1,91,3,4,mANAGEr,1,Married,"16595,0",5626,7,NaN,No,16,"3,0",2,NaN,1,"22,0",2,"3,0",18,NaN,11,8,"16595,0",1982,1000000000$,NaN,NaN,True
9,41,No,travel_frequently,"840,0$",NaN,9,3,NaN,1,"999,0",1,0,64,3,5,reSEaRCH DIrectoR,3,NaN,NaN,3735,2,NaN,No,17,"3,0",2,NaN,1,"21,0",2,"4,0",18,NaN,0,11,NaN,1982,1000000000$,NaN,NaN,0


In [52]:
#FASE 1.
#Creamos la función para leer el archivo csv.
def lectura_archivo(csv):
    data = pd.read_csv(csv, index_col= 0)
    return data

In [4]:
# lectura_archivo('HR_RAW_DATA.csv')

In [53]:
#Creamos la función para explorar los datos. 

def exploracion_datos(df):
    print('_____________ INFORMACIÓN GENERAL DEL DATAFRAME ____________\n')
    print(df.info())

    print('___________________ FORMA DEL DATAFRAME ____________________\n')
    
    print(f"El número de filas que tenemos es de {df.shape[0]}.\nEl número de columnas es de {df.shape[1]}\n")
    

    print('_______________ NULOS, ÚNICOS Y DUPLICADOS _________________\n')
    
    print('La cantidad de valores NULOS por columna es de:\n')
    print(df.isnull().sum())
    print('____________________________________________________________\n')

    print('La cantidad de valores ÚNICOS por columna es de:\n')
        
    for columna in df.columns:
        cantidad_valores_unicos = len(df[columna].unique())
    
        print(f'La columna {columna}: {cantidad_valores_unicos}')

    """ Otra forma más rápida de obtener la lista de valores únicos por columna es usando df.nunique()"""

    print('____________________________________________________________\n')

    print('La cantidad de valores DUPLICADOS por columna es de:\n')

    """En análisis posteriores hemos detectado que hay columnas con valores duplicados que nos interesa filtrar, 
    así que vamos a realizar otro bucle for para iterar por todas las columnas del DF y obtener los duplicados de cada una de ellas."""

    for columna in df.columns:
        cantidad_duplicados = df[columna].duplicated().sum()
    
        print(f'La columna {columna}: {cantidad_duplicados}')


    print('____________________ RESUMEN ESTADÍSTICO ____________________')
    print('____________________ Variables Numéricas __________________\n')
    print(df.describe().T)
    
    print('___________________ Variables Categóricas _________________\n')
    print(df.describe(include='object').T)

    


In [54]:
exploracion_datos(df_HR)

_____________ INFORMACIÓN GENERAL DEL DATAFRAME ____________

<class 'pandas.core.frame.DataFrame'>
Index: 1614 entries, 0 to 1613
Data columns (total 41 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Age                       1614 non-null   object 
 1   Attrition                 1614 non-null   object 
 2   BusinessTravel            842 non-null    object 
 3   DailyRate                 1614 non-null   object 
 4   Department                302 non-null    object 
 5   DistanceFromHome          1614 non-null   int64  
 6   Education                 1614 non-null   int64  
 7   EducationField            869 non-null    object 
 8   employeecount             1614 non-null   int64  
 9   employeenumber            1183 non-null   object 
 10  EnvironmentSatisfaction   1614 non-null   int64  
 11  Gender                    1614 non-null   int64  
 12  HourlyRate                1614 non-null   object 
 13  JobInv

In [55]:
#FASE 2
""" ¿Qué hacemos con la columna del id de empleados que tiene duplicados? """
df_HR['employeenumber'].duplicated().sum()

534

In [56]:
#employeecount: esta columna tiene el mismo valor para todas las filas, por lo que es redundante y se puede eliminar.¿¿??
df_HR['employeecount'].unique()

array([1])

In [57]:
#Eliminamos la columna "employeecount"
df_HR.drop(columns='employeecount', inplace= True)

In [58]:
#NUMBERCHILDREN: Tiene solo valores nulos, se puede eliminar.
df_HR['NUMBERCHILDREN'].isnull().sum()

1614

In [59]:
#Eliminamos la columna "NUMBERCHILDREN"
df_HR.drop(columns='NUMBERCHILDREN', inplace= True)

In [60]:
#SameAsMonthlyIncome: parece un duplicado de la columna MonthlyIncome,se puede eliminar. 
df_HR[['SameAsMonthlyIncome', 'MonthlyIncome', 'MonthlyRate', 'HourlyRate']]

,SameAsMonthlyIncome,MonthlyIncome,MonthlyRate,HourlyRate
0,"19537,0","19537,0",6462,51
1,"19999,0","19999,0",5678,65
2,"19232,0","19232,0",4933,58
3,"17169,0","17169,0",26703,82
4,NaN,NaN,7739,45
...,...,...,...,...
1609,"10325,0","10325,0",5518,51
1610,"4447,0","4447,0",23163,57
1611,NaN,NaN,2560,41
1612,NaN,NaN,25952,84


In [61]:
#Eliminamos la columna
df_HR.drop(columns='SameAsMonthlyIncome', inplace= True)
#Buscamos nombre columnas para comprobar que la hemos eliminado.

In [62]:
#Salary: Tiene un solo valor repetido, por lo que se puede eliminar también.
df_HR['Salary'].unique()

array(['1000000000$'], dtype=object)

In [63]:
#Eliminamos la columna "Salary"
df_HR.drop(columns='Salary', inplace= True)

In [64]:
#Reemplazamos los valores 1 y 0 de la columna Gender por "M" de masculino y "F" de femenino, usando loc.

df_HR.loc[df_HR['Gender'] == 0, 'Gender'] = 'M'
df_HR.loc[df_HR['Gender'] == 1, 'Gender'] = 'F'

print(df_HR['Gender'])


0       M
1       M
2       M
3       F
4       F
       ..
1609    M
1610    M
1611    M
1612    M
1613    M
Name: Gender, Length: 1614, dtype: object


/var/folders/sg/x17x70c54nq9kk2ks_m66x280000gp/T/ipykernel_7909/3872921605.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'M' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_HR.loc[df_HR['Gender'] == 0, 'Gender'] = 'M'


### Función para cambiar a FLOAT

In [65]:
# Creamos la función 'cambiar comas'

lista_tofloat = ['DailyRate', 'HourlyRate', 'MonthlyIncome']

def cambiar_comas(cadena):
     # Remover el símbolo $, reemplazar comas por puntos
     # Usar expresiones regulares para mantener solo los números y puntos
    try:
    # Remover cualquier caracter que no sea dígito o coma
        cleaned_string = re.sub(r'[^\d,]', '', cadena)
        # Reemplazar comas por puntos
        cleaned_string = cleaned_string.replace(",", ".")
        # Convertir a float
        return float(cleaned_string)
    except:
        return np.nan # Devuelve un np.nan si hay un valor inválido

# Aplicar la función a cada columna en la lista
for col in lista_tofloat:
    df_HR[col] = df_HR[col].apply(cambiar_comas)


# Comprobamos que la columna, que antes era de tipo string, ahora es float64.
df_HR[lista_tofloat].dtypes



DailyRate        float64
HourlyRate       float64
MonthlyIncome    float64
dtype: object

In [66]:
df_HR.head(10)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,employeenumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NUMCOMPANIESWORKED,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TOTALWORKINGYEARS,TrainingTimesLastYear,WORKLIFEBALANCE,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YEARSWITHCURRMANAGER,DateBirth,RoleDepartament,RemoteWork
0,51,No,NaN,684.0,NaN,6,3,NaN,"162,0",1,M,51.0,3,5,resEArch DIREcToR,3,NaN,19537.0,6462,7,Y,No,13,"3,0",3,NaN,0,NaN,5,"3,0",20,NaN,15,15,1972,NaN,Yes
1,52,No,NaN,699.0,NaN,1,4,Life Sciences,"259,0",3,M,65.0,2,5,ManAGeR,3,NaN,19999.0,5678,0,NaN,NaN,14,"3,0",1,NaN,1,"34,0",5,"3,0",33,NaN,11,9,1971,NaN,1
2,42,No,travel_rarely,532.0,Research & Development,4,2,Technical Degree,"319,0",3,M,58.0,3,5,ManaGER,4,Married,19232.0,4933,1,NaN,No,11,"3,0",4,NaN,0,"22,0",3,NaN,22,NaN,11,15,1981,ManaGER - Research & Development,1
3,47,No,travel_rarely,359.0,NaN,2,4,Medical,NaN,1,F,82.0,3,4,ReseArCH DIrECtOr,3,Married,17169.0,26703,3,Y,NaN,19,"3,0",2,NaN,2,NaN,2,NaN,20,NaN,5,6,1976,NaN,False
4,46,No,NaN,1319.0,NaN,3,3,Technical Degree,NaN,1,F,45.0,4,4,sAleS EXECUtIve,1,Divorced,NaN,7739,2,Y,No,12,"3,0",4,NaN,1,NaN,5,"3,0",19,NaN,2,8,1977,NaN,0
5,48,No,NaN,117.0,Research & Development,22,3,Medical,"1900,0",4,F,58.0,3,4,MANAger,4,NaN,17174.0,2437,3,NaN,No,11,"3,0",2,NaN,1,NaN,3,"3,0",22,NaN,4,7,1975,MANAger - Research & Development,Yes
6,59,No,NaN,1435.0,NaN,25,3,Life Sciences,"81,0",1,F,99.0,3,3,Sales ExeCutIVe,1,NaN,NaN,2354,7,Y,NaN,11,"3,0",4,NaN,0,"28,0",3,"2,0",21,NaN,7,9,1964,NaN,True
7,42,No,travel_rarely,635.0,NaN,1,1,NaN,"387,0",2,M,99.0,3,2,Sales eXEcUTiVe,3,Married,NaN,24532,1,NaN,No,25,"4,0",3,NaN,0,"20,0",3,"3,0",20,NaN,11,6,1981,NaN,0
8,41,No,NaN,1276.0,NaN,2,5,NaN,NaN,2,F,91.0,3,4,mANAGEr,1,Married,16595.0,5626,7,NaN,No,16,"3,0",2,NaN,1,"22,0",2,"3,0",18,NaN,11,8,1982,NaN,True
9,41,No,travel_frequently,840.0,NaN,9,3,NaN,"999,0",1,M,64.0,3,5,reSEaRCH DIrectoR,3,NaN,NaN,3735,2,NaN,No,17,"3,0",2,NaN,1,"21,0",2,"4,0",18,NaN,0,11,1982,NaN,0


In [67]:
#lo guardo para que se guarden los datos

df_HR.to_csv("df_HR_limpio.csv", index = False)

### Función para cambiar a INT

In [68]:
# Creamos la función 'cambiar comas'

lista_toint = ['WORKLIFEBALANCE']

def cambiar_comas(cadena):
     # Remover el símbolo $, reemplazar comas por puntos
     # Usar expresiones regulares para mantener solo los números y puntos
    try:
    # Remover cualquier caracter que no sea dígito o coma
        cleaned_string = re.sub(r'[^\d,]', '', cadena)
        # Reemplazar comas por puntos
        cleaned_string = cleaned_string.replace(",", ".")
        # Convertir a float
        return int(cleaned_string)
    except:
        return np.nan # Devuelve un np.nan si hay un valor inválido

# Aplicar la función a cada columna en la lista
for col in lista_toint:
    #convertir a int
    df_HR[col] = df_HR[col].apply(cambiar_comas).astype('Int64')


# Comprobamos que la columna, que antes era de tipo string, ahora es float64.
df_HR[lista_toint].dtypes

WORKLIFEBALANCE    Int64
dtype: object

In [69]:
#pasar a csv para que se guarde

df_HR.to_csv('df_HR_limpio.csv')

In [72]:
df_HR = pd.read_csv('df_HR_limpio.csv')
df_HR.sample(10)

,Unnamed: 0,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,employeenumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NUMCOMPANIESWORKED,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TOTALWORKINGYEARS,TrainingTimesLastYear,WORKLIFEBALANCE,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YEARSWITHCURRMANAGER,DateBirth,RoleDepartament,RemoteWork
688,688,28,No,NaN,1117.0,NaN,8,2,NaN,NaN,4,F,66.0,3,1,rEseaRcH scieNtisT,4,NaN,NaN,4488,1,NaN,No,21,"4,0",4,"80,0",0,NaN,3,NaN,5,NaN,0,2,1995,NaN,1
527,527,29,No,travel_rarely,665.0,NaN,15,3,NaN,"346,0",3,M,60.0,3,1,reSearch sCienTiSt,4,NaN,NaN,22673,1,NaN,No,19,"3,0",1,NaN,0,"6,0",1,NaN,6,NaN,1,5,1994,NaN,False
1367,1367,38,No,travel_rarely,130.0,NaN,2,2,Marketing,"1125,0",4,M,32.0,3,3,SAlEs eXEcUTIVe,2,Single,7351.0,20619,7,Y,No,16,"3,0",3,NaN,0,NaN,2,NaN,1,NaN,0,0,1985,NaN,Yes
1104,1104,44,No,travel_rarely,528.0,Human Resources,1,3,NaN,"1683,0",3,F,44.0,3,1,HuMan resoUrCes,4,NaN,NaN,4167,4,Y,Yes,18,"3,0",1,NaN,3,"8,0",2,NaN,2,NaN,2,2,1979,HuMan resoUrCes - Human Resources,False
583,583,41,No,travel_rarely,483.0,NaN,6,3,NaN,"466,0",4,M,NaN,2,2,MAnUFacTuRING DIREctOR,2,Single,NaN,10110,6,NaN,Yes,15,"3,0",4,NaN,0,NaN,3,NaN,5,NaN,1,2,1982,NaN,Yes
1570,1570,41,No,NaN,509.0,NaN,2,4,Other,"616,0",1,F,62.0,2,2,heaLThCARE reprESeNtATIVE,3,Single,NaN,2112,2,Y,Yes,17,"3,0",1,"80,0",0,"10,0",3,NaN,8,NaN,0,7,1982,NaN,Yes
1422,1422,32,Yes,travel_rarely,1259.0,NaN,2,4,Life Sciences,NaN,4,M,95.0,3,1,LAbOrATORY tEChNiciAn,2,Single,1393.0,24852,1,NaN,No,12,"3,0",1,NaN,0,"1,0",2,NaN,1,NaN,0,0,1991,NaN,1
1428,1428,32,Yes,travel_rarely,964.0,Sales,1,2,NaN,NaN,1,M,NaN,1,2,SalES eXecUTiVE,2,Single,6735.0,12147,6,NaN,NaN,15,"3,0",2,"80,0",0,"10,0",2,NaN,0,NaN,0,0,1991,SalES eXecUTiVE - Sales,0
1221,1221,36,No,travel_frequently,1213.0,NaN,2,1,NaN,"1890,0",2,M,94.0,2,2,hUMaN rESouRces,4,NaN,3886.0,4223,1,NaN,No,21,"4,0",4,"80,0",0,"10,0",2,NaN,10,NaN,0,8,1987,NaN,True
958,958,44,No,travel_rarely,986.0,NaN,8,4,Life Sciences,"874,0",1,M,62.0,4,1,LAbORaToRY TecHNICiAn,4,NaN,NaN,5044,2,NaN,Yes,24,"4,0",3,NaN,1,"10,0",2,NaN,3,NaN,0,2,1979,NaN,False


In [73]:
#Cambiamos el tipo de dato de la columna Age para que sea interger.

df_HR.loc[df_HR['Age'] == 'forty-seven', 'Age'] = '47'
df_HR.loc[df_HR['Age'] == 'fifty-eight', 'Age'] = '58'
df_HR.loc[df_HR['Age'] == 'thirty-six', 'Age'] = '36'
df_HR.loc[df_HR['Age'] == 'fifty-five', 'Age'] = '55'
df_HR.loc[df_HR['Age'] == 'fifty-two', 'Age'] = '52'
df_HR.loc[df_HR['Age'] == 'thirty-one', 'Age'] = '31'
df_HR.loc[df_HR['Age'] == 'thirty', 'Age'] = '30'
df_HR.loc[df_HR['Age'] == 'twenty-six', 'Age'] = '26'
df_HR.loc[df_HR['Age'] == 'thirty-seven', 'Age'] = '37'
df_HR.loc[df_HR['Age'] == 'thirty-two', 'Age'] = '32'
df_HR.loc[df_HR['Age'] == 'thirty-seven', 'Age'] = '37'
df_HR.loc[df_HR['Age'] == 'twenty-four', 'Age'] = '24'

df_HR['Age']= df_HR['Age'].astype(int)
df_HR['Age'].dtype

dtype('int64')

In [74]:
#Buscamos los valores negativos de la columna 'DistanceFromHome'
valores_negativos = df_HR[df_HR['DistanceFromHome'] < 0]['DistanceFromHome']
print(valores_negativos)
print('_________________________________________________')
print(f'Hay {valores_negativos.shape[0]} filas con valores negativos')

18     -13
24     -42
30     -37
41     -35
57     -26
        ..
1572   -29
1576   -24
1590   -35
1601   -43
1611   -13
Name: DistanceFromHome, Length: 192, dtype: int64
_________________________________________________
Hay 192 filas con valores negativos


In [75]:
#Sustituimos valores negativos por positivos
df_HR.loc[df_HR['DistanceFromHome'] < 0, 'DistanceFromHome'] = df_HR['DistanceFromHome'].abs()

#Comprobamos que ya no hay valores negativos
df_HR[df_HR['DistanceFromHome'] < 0]

,Unnamed: 0,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,employeenumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NUMCOMPANIESWORKED,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TOTALWORKINGYEARS,TrainingTimesLastYear,WORKLIFEBALANCE,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YEARSWITHCURRMANAGER,DateBirth,RoleDepartament,RemoteWork


In [76]:
#Corregimos errores tipográficos en la columna MaritalStatus... 
df_HR['MaritalStatus'].unique()

array([nan, 'Married', 'Divorced', 'Single', 'divorced', 'Marreid'],
      dtype=object)

In [77]:
#Como 'Marreid'.
df_HR.loc[df_HR['MaritalStatus'] == 'Marreid', 'MaritalStatus'] = 'Married'
df_HR['MaritalStatus'].unique()

#O 'divorced'.
df_HR.loc[df_HR['MaritalStatus'] == 'divorced', 'MaritalStatus'] = 'Divorced'
df_HR['MaritalStatus'].unique()

array([nan, 'Married', 'Divorced', 'Single'], dtype=object)

In [ ]:
""" Mirar qué otras columnas podemos cambiar de typo porque no estén en el correcto, como NUMBERCHILDRENS.
Ver si alguna otra columna tiene muchos nulos y no nos interesa (de cara a eliminarla). """

' Mirar qué otras columnas podemos cambiar de typo porque no estén en el correcto, como NUMBERCHILDRENS.\nVer si alguna otra columna tiene muchos nulos y no nos interesa (de cara a eliminarla). '

In [ ]:
#FASE 4
#Seudocódigo.
""" Usar como filtro los valores de la columna **Attrition**. Los valores == YES han abandonado la empresa. 
    Buscar su nivel de Satisfacción. Columnas Age, Gender, EnviromentSatisfaction, JobInvolvement, 
    JobSatisfaction, 'RelationshipSatisfaction, WORKLIFEBALANCE, YearsSinceLastPromotion, ¿ YEARSWITHCURRMANAGER ?, 
    Salary, RoleDepartament, NUMBERCHILDREN, DistanceFromHome - RemoteWork. """

" Usar como filtro los valores de la columna **Attrition**. Los valores == YES han abandonado la empresa. \n    Buscar su nivel de Satisfacción. Columnas Age, Gender, EnviromentSatisfaction, JobInvolvement, \n    JobSatisfaction, 'RelationshipSatisfaction, WORKLIFEBALANCE, YearsSinceLastPromotion, ¿ YEARSWITHCURRMANAGER ?, \n    Salary, RoleDepartament, NUMBERCHILDREN, DistanceFromHome - RemoteWork. "

In [78]:
df_HR.to_csv('df_HR_limpio.csv', index=False)

In [79]:
#ABRO el nuevo archivo, el limpio para seguir trabajando

df_limpio = pd.read_csv('df_HR_limpio.csv', index_col= 0)
df_limpio.head(2)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,employeenumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NUMCOMPANIESWORKED,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TOTALWORKINGYEARS,TrainingTimesLastYear,WORKLIFEBALANCE,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YEARSWITHCURRMANAGER,DateBirth,RoleDepartament,RemoteWork
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,51,No,NaN,684.0,NaN,6,3,NaN,"162,0",1,M,51.0,3,5,resEArch DIREcToR,3,NaN,19537.0,6462,7,Y,No,13,"3,0",3,NaN,0,NaN,5,NaN,20,NaN,15,15,1972,NaN,Yes
1,52,No,NaN,699.0,NaN,1,4,Life Sciences,"259,0",3,M,65.0,2,5,ManAGeR,3,NaN,19999.0,5678,0,NaN,NaN,14,"3,0",1,NaN,1,"34,0",5,NaN,33,NaN,11,9,1971,NaN,1


In [80]:
# Función para mapear los valores a 'yes' y 'no', RemoteWork
def map_values(value):
    if str(value).lower() in ['1', 'true', 'yes']:
        return 'yes'
    elif str(value).lower() in ['0', 'false', 'no']:
        return 'no'
    else:
        return value  

# Aplicar la función a la columna
df_limpio['RemoteWork'] = df_limpio['RemoteWork'].apply(map_values)



In [81]:
df_limpio.sample(5)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,employeenumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NUMCOMPANIESWORKED,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TOTALWORKINGYEARS,TrainingTimesLastYear,WORKLIFEBALANCE,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YEARSWITHCURRMANAGER,DateBirth,RoleDepartament,RemoteWork
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
464,46,Yes,NaN,NaN,NaN,10,3,Life Sciences,"1869,0",3,F,64.0,3,3,saLES exeCuTIVe,2,NaN,7314.0,14011,5,Y,NaN,21,"4,0",3,"80,0",3,"14,0",2,NaN,8,NaN,0,7,1977,NaN,yes
848,29,Yes,NaN,992.0,NaN,1,3,NaN,NaN,3,M,85.0,3,1,ResEARCh SCIENtIsT,3,Single,NaN,19757,0,NaN,NaN,14,"3,0",4,NaN,0,NaN,1,NaN,6,NaN,1,5,1994,NaN,no
1389,45,No,travel_frequently,364.0,NaN,25,3,Medical,"1306,0",2,F,NaN,3,5,manAGEr,2,NaN,18061.0,13035,3,NaN,NaN,22,"4,0",3,"80,0",0,"22,0",4,NaN,0,NaN,0,0,1978,NaN,no
1121,34,No,travel_rarely,511.0,NaN,3,2,NaN,NaN,4,F,32.0,1,2,sAlES exECuTivE,4,NaN,NaN,25353,5,NaN,No,12,NaN,1,NaN,0,NaN,3,NaN,2,NaN,2,2,1989,NaN,yes
437,45,No,NaN,1015.0,NaN,5,5,NaN,"1611,0",3,F,50.0,1,2,laBORATOrY TEchNICian,1,NaN,5769.0,23447,1,Y,NaN,14,"3,0",1,NaN,0,"10,0",3,NaN,10,NaN,1,4,1978,NaN,no


In [82]:
df_HR["JobRole"] = df_HR["JobRole"].str.title()
df_HR["RoleDepartament"] = df_HR["RoleDepartament"].str.title()
df_HR.rename(columns={'TOTALWORKINGYEARS': 'TotalWorkingYears'}, inplace=True)
df_HR.rename(columns={'employeenumber': 'EmployeeNumber'}, inplace=True)
df_HR.rename(columns={'NUMCOMPANIESWORKED': 'NumCompaniesWorked'}, inplace=True)
df_HR.rename(columns={'WORKLIFEBALANCE': 'WorkLifeBalance'}, inplace=True)
df_HR.rename(columns={'YEARSWITHCURRMANAGER': 'YearsWithCurrManager'}, inplace=True)

In [83]:
df_HR.sample(10)

,Unnamed: 0,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,DateBirth,RoleDepartament,RemoteWork
894,894,34,No,NaN,1381.0,NaN,4,4,NaN,"523,0",3,F,72.0,3,2,Sales Executive,3,Married,6538.0,12740,9,Y,No,15,"3,0",1,NaN,1,NaN,3,NaN,3,NaN,1,2,1989,NaN,1
920,920,21,No,NaN,1343.0,NaN,22,1,Technical Degree,NaN,3,M,49.0,3,1,Sales Representative,3,NaN,3447.0,24444,1,NaN,NaN,11,NaN,3,NaN,0,NaN,2,NaN,3,NaN,1,2,2002,NaN,True
324,324,48,No,travel_rarely,163.0,NaN,2,5,NaN,"595,0",2,F,37.0,3,2,Sales Executive,4,Married,4051.0,19658,2,NaN,No,14,"3,0",1,NaN,1,"14,0",2,NaN,9,NaN,6,7,1975,NaN,True
1038,1038,31,Yes,NaN,NaN,NaN,22,4,NaN,"1389,0",4,M,98.0,3,2,Manufacturing Director,3,Married,NaN,21057,1,NaN,NaN,15,"3,0",4,NaN,2,"10,0",3,NaN,10,NaN,6,7,1992,NaN,True
1313,1313,33,No,NaN,1216.0,Sales,8,4,Marketing,NaN,3,M,39.0,3,2,Sales Executive,3,Divorced,NaN,20431,0,Y,NaN,12,"3,0",4,"80,0",0,"6,0",3,NaN,5,NaN,1,2,1990,Sales Executive - Sales,0
1570,1570,41,No,NaN,509.0,NaN,2,4,Other,"616,0",1,F,62.0,2,2,Healthcare Representative,3,Single,NaN,2112,2,Y,Yes,17,"3,0",1,"80,0",0,"10,0",3,NaN,8,NaN,0,7,1982,NaN,Yes
1417,1417,18,Yes,travel_frequently,544.0,Sales,3,2,Medical,"1624,0",2,F,70.0,3,1,Sales Representative,4,Single,1569.0,18420,1,NaN,Yes,12,"3,0",3,"80,0",0,"0,0",2,NaN,0,NaN,0,0,2005,Sales Representative - Sales,True
995,995,33,No,travel_rarely,654.0,NaN,5,3,Life Sciences,"1099,0",4,M,34.0,2,3,Healthcare Representative,4,NaN,NaN,21214,4,NaN,No,15,"3,0",3,NaN,1,NaN,2,NaN,3,NaN,1,2,1990,NaN,1
1467,1467,39,No,non-travel,105.0,NaN,9,3,NaN,NaN,4,M,87.0,3,5,Manager,4,NaN,19431.0,15302,2,NaN,NaN,13,"3,0",3,NaN,0,NaN,3,NaN,6,NaN,1,3,1984,NaN,False
849,849,54,No,NaN,1147.0,NaN,3,3,NaN,"303,0",4,F,52.0,3,2,Sales Executive,1,Married,5940.0,17011,2,NaN,NaN,14,"3,0",4,NaN,1,"16,0",4,NaN,6,NaN,0,5,1969,NaN,0


In [84]:
df_HR.to_csv("df_HR_limpio.csv", index = False)